In [1]:
import torch
print(f'PyTorch version: {torch.__version__}')
print("Is CUDA enabled?",torch.cuda.is_available())
print('*'*10)
#!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.1
Is CUDA enabled? True
**********
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4080


In [2]:
from detectron2.engine import DefaultPredictor
#import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.structures import Boxes
from detectron2 import model_zoo
# from PIL import Image 
# import PIL 
import cv2
import numpy as np
import tqdm
import random
import matplotlib.pyplot as plt
import os
import math

In [9]:
# need to register custom data so it can be used for training
from detectron2.data.datasets import register_coco_instances
# Removing registered dataset in case a mistake is made
# DatasetCatalog.remove('barbell_train')
# DatasetCatalog.remove('barbell_val')
# DatasetCatalog.remove('barbell_test')
# format is register_coco_instances("YourTrainDatasetName", {},"path to train.json", "path to train image folder")
# for train, valid, and test
register_coco_instances("barbell_train",{},'Barbell-small-2/train/_annotations.coco.json', 'Barbell-small-2/train' )
register_coco_instances("barbell_val",{},'Barbell-small-2/valid/_annotations.coco.json', 'Barbell-small-2/valid' )
register_coco_instances("barbell_test",{},'Barbell-small-2/test/_annotations.coco.json', 'Barbell-small-2/test' )

AssertionError: Dataset 'barbell_train' is already registered!

In [19]:
barbell_train_metadata = MetadataCatalog.get("barbell_train")

videoPath = 'test_video_lidayin.mp4'
video = cv2.VideoCapture(videoPath)
video.set(cv2.CAP_PROP_POS_FRAMES,512) 
ret, frame = video.read()
visualizer = Visualizer(frame[:, :, ::-1], metadata=barbell_train_metadata, scale=0.5)
cv2.imshow('barbell',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
cfg_obj = get_cfg() # obtain detectron2's default config
cfg_obj.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) # load values from a file specified under Detectron2's official configs/ directory
cfg_obj.MODEL.WEIGHTS ='output/model_final.pth'
cfg_obj.DATASETS.TEST = ("barbell_test", )
cfg_obj.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg_obj.MODEL.ROI_HEADS.NUM_CLASSES = 2 # adjust for number of classes

pred_obj = DefaultPredictor(cfg_obj)
barbell_metadata = MetadataCatalog.get("barbell_test")

[12/15 17:40:14 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/model_final.pth ...


In [ ]:
cfg_key = get_cfg() # obtain detectron2's default config
# may need to download config file locally for greater speed
cfg_key.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")) # load values from a file specified under Detectron2's official configs/ directory
cfg_key.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml") # returns the URL to the model trained using the given config
cfg_key.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # threshold to filter out low-scored bounding boxes
cfg_key.MODEL.DEVICE = 'cuda'  # cpu or cuda
# getting metadata for keypoint detection and deleting thing_classes to set to custom
metadata = MetadataCatalog.get(cfg_key.DATASETS.TRAIN[0]); del metadata.thing_classes
# keypoint detection person is 0 and weights are 1, so setting custom labels to lifter and weights
metadata.thing_classes = ['lifter','weights']
pred_keypt = DefaultPredictor(cfg_key)

In [40]:
outputs = pred_obj(frame)
v = Visualizer(frame[:, :, ::-1], metadata, scale=1.2)
out = v.draw_instance_predictions(outputs['instances'].to("cpu"))
outputs

{'instances': Instances(num_instances=4, image_height=360, image_width=640, fields=[pred_boxes: Boxes(tensor([[387.0726, 265.5885, 477.3191, 357.8558],
         [111.5254, 287.8434, 164.7855, 345.9864],
         [157.9690, 287.8644, 199.4764, 344.9593],
         [240.3044, 270.2670, 324.1308, 350.8048]], device='cuda:0')), scores: tensor([0.9963, 0.9875, 0.9837, 0.9791], device='cuda:0'), pred_classes: tensor([1, 1, 1, 1], device='cuda:0')])}

In [41]:
cv2.imshow('test', out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()